In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse import csr_array
from scipy import special as sp

# Part II

In [ ]:
class FD_schemes():
    def __init__(self, S,r,vol, T, K ,dx, dt,M1 = -400, M2 = 1000, auto = True):
        """
        Finite differences model class for Crank Nicolson
        Inputs:
            -   S: stock price
            -   r: risk-free interest rate
            -   vol: volatility fo the stock % in decimals
            -   T: Time period
            -   K: Strike price
            -   dx: Discretization in X
            -   dt: Discretization in time
            -   M1: Low limit of X, default -400
            -   M2: Upp limit of X, default 1000
            -   auto: If True, executes crank nicholson and finite difference automatically
        """

        self.S = S
        self.r = r
        self.vol = vol
        self.T = T
        self.K = K
        self.dx = dx
        self.dt = dt
        
        self.M1 = M1
        self.M2 = M2

        self.N = int((M1 + M2)/self.dx)
        self.n_steps = int(T/self.dt)

        self.X = np.linspace(-M1,M2,num = self.N)
        self.V = np.maximum(np.exp(self.X)-self.K,0)

        self.data_f = [np.copy(self.V)]
        self.data_c = [np.copy(self.V)]

        self.initialize()
        
        if auto = True:
            self.crank_nicholson()
            self.finite_differences()

    def initialize(self):
        """
        Creates the matrix array's for crank nicolson and FD schemes
        """
        #### FTCS
        f1 = (self.r - (self.vol**2)*1/2)*self.dt/(2*self.dx)
        f2 = 1/2 * (self/vol**2)*(self.dt/(self.dx**2))
        df = [np.full(self.N -1, -f1 + f2), np.full(self.N, 1 - 2*f2), np.full(self.N -1, f1 + f2)]

        self.F = diags(df, [-1,0,1]).toarray()
        #### Crank Nicolson
        c1 = (self.r - (self.vol**2)*1/2)*self.dt/(4*self.dx)
        c2 = 1/4 * (self/vol**2)*(self.dt/(self.dx**2))
        c3 =  self.r*self.dt/2
        
        da = [np.full(self.N -1, -c1 + c2), np.full(self.N, 1 - 2*c2 - c3), np.full(self.N -1, c1 + c2)] 
        db = [np.full(self.N -1, c1 - c2), np.full(self.N, 1 + 2*c2 + c3), np.full(self.N -1, - c1 - c2)] 
        
        self.A = diags(da, [-1,0,1]).toarray()
        self.B = diags(db , [-1,0,1]).toarray()
    
    def crank_nicolson(self):
        """
        Does the Crank Nicholson scheme for n_steps
        """
        B_inv = np.linalg.inv(self.B)
        
        for _ in range(self.n_steps):
            V_d = B_inv @ self.A @ self.V
            self.V = V_d
            self.data_c += [np.copy(self.V)]
        
        self.deltas = np.diff(self.V)/np.diff(np.exp(self.X))
        
        ## Reset
        self.V = np.maximum(np.exp(self.X)-self.K),0)

    def finite_differences(self):
        """
        Does the Fintie difference schemge scheme for n_steps
        """
        
        for _ in range(self.n_steps):
            V_d = self.F @ self.V
            self.V = V_d
            self.data_f += [np.copy(self.V)]
        
        self.deltas = np.diff(self.V)/np.diff(np.exp(self.X))

        ## Reset
        self.V = np.maximum(np.exp(self.X)-self.K,0)

    